In [1]:
import pandas as pd
df_treino = pd.read_csv('../input/competicao-um-ic/train.csv')
df_teste = pd.read_csv('../input/competicao-um-ic/test.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# ordinal encoder possui essa função, mas a versão do colab não está atualizada
def fillna(df):
    for col in df:
        if df[col].dtype == 'O':
            df[col].fillna('NA', inplace=True)
        else:
            df[col].fillna(999999, inplace=True)

In [3]:
fillna(df_treino)
fillna(df_teste)

In [4]:
x_treino = df_treino.iloc[:, :-1]
y_treino = df_treino.iloc[:, -1]
# X teste é df_teste

In [5]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

x_treino = pd.DataFrame(enc.fit_transform(x_treino))

In [6]:
x_teste = pd.DataFrame(enc.transform(df_teste))

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

select = SelectKBest(chi2, k=19)
select.fit_transform(x_treino, y_treino)
cols = select.get_support()
x_treino_novo = x_treino.iloc[:,cols]

In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_features=10, max_depth=20)
clf.fit(x_treino_novo, y_treino)

RandomForestClassifier(max_depth=20, max_features=10)

In [9]:
x_teste_novo = x_teste.iloc[:, cols]

In [10]:
predicao = clf.predict(x_teste_novo)

In [11]:
ID = df_teste.iloc[:, 0].values.tolist()
my_submission = pd.DataFrame({'ID': ID ,'DS_STATUS_ITEM': predicao})
my_submission.to_csv('submission.csv', index=False)